<a href="https://colab.research.google.com/github/2kunal6/UniBonn/blob/master/cuda_lab/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from torchvision import datasets, transforms
import numpy as np
from tqdm import tqdm

train_mnist = datasets.MNIST("", train=True, transform=transforms.ToTensor(), download=True)

test_mnist = datasets.MNIST("", train=False, transform=transforms.ToTensor(), download=True)

train = []
test = []
for data in train_mnist:
  train.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])

for data in test_mnist:
  test.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])


print(len(train))
print(len(test))
print(len(train[0][0]), type(train[0][0]))

60000
10000
784 <class 'numpy.ndarray'>


In [48]:
np.random.seed(1)
num_input = 784
num_output = 10
weight = np.random.random((num_output, num_input))
bias = 0.0
learning_rate = 0.01

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def forward(x, y):
  x = data[0]
  y = data[1]
  net = [0] * num_output
  for i in range(num_output):
    for j in range(num_input):
      net[i] += (x[j] * weight[i][j] + bias)
  return softmax(net)

# MSE
def calc_loss(y, y_pred):
  #print(y)
  #print(y_pred)
  return ((y-y_pred)**2).mean()

# gradient of MSE dJ/dw = 1/N 2x (w*x - y)
def calc_gradient(x, y, y_pred):
  grad_list = []
  for i in range(len(y)):
    grad_list.append((np.dot(2*x, y_pred[i] - y[i])).mean())
  return grad_list

#print("prediction before training : ", forward(10))

for epoch in range(15):
  BATCH_SIZE = 20 # without replacement
  grad_list_batch = [0]*num_output
  num_iter=0
  for data in train:
    num_iter+=1
    x, y = data[0], data[1]
    y_pred = forward(x, y)
    #print(sum(y_pred))
    loss = calc_loss(y, y_pred)

    grad = calc_gradient(x, y, y_pred)

    grad_list_batch = [sum(x) for x in zip(grad_list_batch, grad)]

    if(num_iter%BATCH_SIZE==0):
      grad_list_batch = [x/BATCH_SIZE for x in grad_list_batch]
      for i in range(num_output):
        for j in range(num_input):
          weight[i][j]-=(learning_rate*grad_list_batch[i])
    
    print(loss)
    
  break
  w -= learning_rate * grad

  print("epoch : ", epoch, "weight : ", w, "loss : ", loss)

0.13541054446762196
0.01106475443027449
0.0923564896888372
0.12121184297869461
0.05692452080420406
0.14654895697209333
0.040980810746827495
0.17959633991601529
0.07102751867533576
0.14379530245820996
0.1939294983536079
0.1287536128398447
0.1947083012927825
0.11826929994353179
0.14145672200633447
0.12874793430532874
0.0738063216792662
0.1293812186501319
0.12527593880240445
0.14243921452056624
0.19246277514926075
0.06143558159594699
0.17429320526270342
0.11912227062430694
0.11720654865168871
0.15215093972020827
0.17415149593876256
0.1612683295297299
0.16010376079483635
0.11255422650216426
0.14530246961374632
0.13577009866021889
0.09120619418625528
0.12252836521385904
0.18309411913427237
0.16818802365342506
0.1240769114782437
0.1958507982254252
0.13654082632959466
0.1051192450656192
0.05199327140876627
0.16176241475232783
0.12266119126457711
0.1216564535256381
0.13585230072753612
0.12734054590955765
0.1502495918590551
0.12593398186903793
0.10622337519782218
0.1897328375347497
0.1512608886

KeyboardInterrupt: ignored